Agora outros problemas foram destacados também que vão afetar em nossa análise na busca por *churn's*. Estas são:

- Durante 2011, houve uma predominância agressiva de cupons, em relação a 2010 que quase não houve;

- Em 2011, para manter o mesmo volume de vendas, em relação a 2010, teve-se que injetar muito mais dinheiro em cupons e marketing;

- Aumento expressivo de clientes lotou as lojas em 2011 afetando clientes campeões;

- Troca de funcionários no meio de 2011 por estresse de funcionários durante o ano;

- Diversificação de fornecedores e barateamento de produtos.

# **Introdução**